In [192]:
import warnings
warnings.filterwarnings('ignore')

In [257]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import time
import random
from selenium.webdriver.chrome.options import Options
import pandas as pd
from tqdm import tqdm

In [194]:
BASE_URL = 'http://saha.moi.ir/'
USERNAME = '1741995108'
PASSWORD = '4NpzuFqdEyCDv6T'

# USERNAME = '1270609726'
# PASSWORD = '093678900450'

In [195]:
def random_comment():
    comments = [
        'درود',
        'سپاس',
        'متشکرم',
        'خداقوت',
        '✔️',
        '🙏🏻',
        '🙏🏻🙏🏻',
        '🙏🏻🙏🏻🙏🏻',
        '✅️',
        '👍🏻',
        '👍🏻👍🏻',
        '👍🏻👍🏻👍🏻',
        'تشکر',
        'درود بر شما',
        'ممنون',
        '👌',
        'تشکر از ارائه مطالب ارزنده شما',
        ' ممنون از مطالب خوبتون',
        'آموزنده بود',
        'سپاس',
        'احسنت',
        'جالب بود',
        'ممنون از اشتراک گذاری'
]
    _ = ''
    for i in range(random.choice([1,2])):
        selected = random.choice(comments).strip()
        if selected == _.strip():
            continue
        _+= f' {selected}'
        _ = _.strip()
    return _
random_comment()

'تشکر 🙏🏻🙏🏻'

In [196]:
def create_driver(headless = True):
    options = Options()
    options.headless = headless
    options.add_argument('ignore-certificate-errors')
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    options.add_experimental_option("prefs",prefs)
    driver = webdriver.Chrome("/usr/bin/chromedriver", options=options)
    driver.set_window_size(700, 800)
    driver.set_window_position(900, 100, windowHandle ='current')
    return driver

In [197]:
def auto_login():
    global driver
    global BASE_URL
    driver.get(BASE_URL)
    driver.find_element(By.ID, "login").send_keys(USERNAME)
    driver.find_element(By.ID, "password").send_keys(PASSWORD)
    driver.find_element(By.ID, "loginbtn").click()
    print('successfully logined ...')

In [303]:
def data_load():
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        time.sleep(5)

        new_height = driver.execute_script("return document.body.scrollHeight")

        if new_height == last_height:
            break
        last_height = new_height 

In [199]:
def search(ID, posts):
    return [element for element in posts if element['id'] == ID]

In [318]:
def write_comments(post):
    try:
        global base_window;
        global driver
        global comments_count
        global reset_comments
        global posts
        driver.execute_script("window.open();")
        driver.switch_to.window(driver.window_handles[1]) 
        driver.get(post['href'])


        tumbs = driver.find_elements(By.CLASS_NAME,"js_like_link_like")
        for tumb in tumbs:
            if comments_count==reset_comments:
                print('comments_count reset fire ....')
                
                base_window  = None
                comments_count = 0
                reset_comments = RESET_COMMENT_COUNT
                posts = None
                return get_score()
            
            try:
                unlike = tumb.find_element(
                By.XPATH, "following-sibling::*[1]"
                    ).get_attribute('style')
                if unlike == 'display: none;':
                    tumb.click()
                    comments_count +=1
                    print(f'*comment_{comments_count}...')
            except Exception as err:
                display(err)
                continue
        
        
        driver.close()
        driver.switch_to.window(base_window)
    except Exception:
        pass

In [342]:
def collect_post():
    global base_window
    global comments_count
    global posts
    global driver
    global reset_comments    
    global RESET_COMMENT_COUNT
    global MAX_UNUSED_POST_COUNT
#     print(f'post_count: {len(posts)}')
    try:
        new = driver.find_elements(By.CLASS_NAME, "js_feed_view_more_entry_holder")

        if (len(new) == len(posts)):
            driver.find_element(By.CSS_SELECTOR, "#feed_view_more > a").click()
            time.sleep(15)
            collect_post()
            
        for post in new:
            ID   = post.get_attribute('id')

            if len(posts[posts['id']==ID]):
                continue

            href = post.find_element(By.CLASS_NAME,'feed_permalink').get_attribute('href')
            _ = pd.Series({
                "id": ID,
                "element" : post,
                "href" : href,
                "done" : False
            })
            posts = posts.append(_ ,ignore_index=True)
    
            write_comments(posts.iloc[-1])
            posts.iloc[-1]['done'] = True
            
            
            
        print(f'**post_count: {len(new)}')
        
        
        if len(new) == MAX_UNUSED_POST_COUNT and comments_count <= RESET_COMMENT_COUNT:
            print('reseting ...')
#             driver = None
            base_window  = None
            comments_count = 0
            reset_comments = RESET_COMMENT_COUNT
            posts = None 
            return get_score()
        
#         print('data loaded ....')
        data_load()        
        return collect_post()
    except Exception as err:
        display(err)
        collect_post()

In [343]:
def get_score():
    global base_window
    global comments_count
    global posts
    global driver
    global reset_comments    
    global RESET_COMMENT_COUNT
    try:
        
        driver.close()
        driver = create_driver(False)
        auto_login()
        base_window = driver.current_window_handle
        comments_count = 0
        posts = pd.DataFrame(columns=['id','element','href','done'])

        collect_post()
        print('driver closed ...')
        driver.close()
    except Exception as err:
        display(err)
        driver = None
        base_window  = None
        comments_count = 0
        reset_comments = RESET_COMMENT_COUNT
        posts = None 
        get_score()
        

In [344]:
RESET_COMMENT_COUNT = 20
MAX_UNUSED_POST_COUNT = 10
driver = None
base_window  = None
comments_count = 0
reset_comments = RESET_COMMENT_COUNT
posts = None 

driver = create_driver(False)
print('starting ...')

get_score()

starting ...
successfully logined ...
**post_count: 20
**post_count: 60
**post_count: 80
**post_count: 100
**post_count: 120
**post_count: 140
**post_count: 160
**post_count: 180


KeyboardInterrupt: 